# Cruce por Similitud

In [21]:
import fuzzywuzzy
from fuzzywuzzy import fuzz, process

In [22]:
palabra_a = 'FuzzyWuzzy es lo mejor'
palabra_b = 'fuzzy wuzzy lo mejor es'

In [23]:
fuzz.ratio(palabra_a,palabra_b)

76

In [24]:
fuzz.ratio(palabra_a.lower(),palabra_b.lower())

84

FuzzyWuzzy también tiene funciones de token que tokenizan las cadenas, cambian las mayúsculas a minúsculas y eliminan la puntuación. La función **token_sort_ratio()** ordena las cadenas alfabéticamente y luego las une. Luego, se calcula fuzz.ratio(). Esto puede ser útil cuando las cadenas que está comparando tienen la misma ortografía pero no están en el mismo orden.

In [25]:
palabra_a1 = 'Gilberto Almonte Frias'
palabra_a2 = 'Almonte Frias, Gilberto'

In [26]:
fuzz.ratio(palabra_a1,palabra_a2)

58

In [27]:
fuzz.token_sort_ratio(palabra_a1,palabra_a2)

100

In [28]:
opciones = ['gilberto alm', 
            'Gilberto Almonte',
            'GIlberto Almonte Frias',
            'G. Almonte',
            'Gilberto A.']

process.extract('Gilbert', opciones, scorer=fuzz.token_sort_ratio)

[('Gilberto A.', 82),
 ('gilberto alm', 74),
 ('Gilberto Almonte', 61),
 ('GIlberto Almonte Frias', 48),
 ('G. Almonte', 25)]

In [29]:
process.extractOne('Gilbert', opciones, scorer=fuzz.token_sort_ratio)

('Gilberto A.', 82)

In [30]:
df_base1 = pd.read_excel('Base1.xlsx')
df_base2 = pd.read_excel('Base2.xlsx')

In [31]:
df_base1.head()

,Nombre
0,Gilberto Almonte
1,Luis Perez
2,Maria Gomez
3,Maria A. Perez
4,Alberto Gonzales


In [32]:
df_base2.head()

,Nombre,Edad
0,"Almonte, Gilberto",26
1,"Perez, Luis",21
2,"Gomez, Maria",30
3,"Perez, Maria Alejandra",22
4,"Gonzales, Alberto",30


In [33]:
lista_base1 = list(df_base1['Nombre'])
lista_base2 = list(df_base2['Nombre'])

In [34]:
def evaluador(lista1,lista2):
    lista_nombre = []
    lista_ratios = []
    for valor1 in lista1:
        if valor1 in lista2:
            lista_nombre.append(valor1)
            lista_ratios.append(100)
        else:
            x = process.extractOne(valor1, 
                                   lista2, 
                                   scorer=fuzz.token_sort_ratio)
            lista_nombre.append(x[0])
            lista_ratios.append(x[1])
    return lista_nombre,lista_ratios

In [36]:
nombre, ratio = evaluador(lista_base1,lista_base2)

In [37]:
nombre

['Almonte, Gilberto',
 'Perez, Luis',
 'Gomez, Maria',
 'Perez, Maria Alejandra',
 'Gonzales, Alberto',
 'Martinez Lara, Luis Manuel',
 'Enrique, Pedro']

In [38]:
df_resultado = pd.DataFrame()
df_resultado['Base1'] = df_base1['Nombre']
df_resultado['CoincidenciaBase2'] = nombre
df_resultado['Ratio'] = ratio

In [39]:
df_resultado

,Base1,CoincidenciaBase2,Ratio
0,Gilberto Almonte,"Almonte, Gilberto",100
1,Luis Perez,"Perez, Luis",100
2,Maria Gomez,"Gomez, Maria",100
3,Maria A. Perez,"Perez, Maria Alejandra",76
4,Alberto Gonzales,"Gonzales, Alberto",100
5,Luis Manuel Martinez L.,"Martinez Lara, Luis Manuel",94
6,Pedro Enrique,"Enrique, Pedro",100


In [40]:
df_resultado.merge(df_base2, 
                   left_on='CoincidenciaBase2', 
                   right_on='Nombre')

,Base1,CoincidenciaBase2,Ratio,Nombre,Edad
0,Gilberto Almonte,"Almonte, Gilberto",100,"Almonte, Gilberto",26
1,Luis Perez,"Perez, Luis",100,"Perez, Luis",21
2,Maria Gomez,"Gomez, Maria",100,"Gomez, Maria",30
3,Maria A. Perez,"Perez, Maria Alejandra",76,"Perez, Maria Alejandra",22
4,Alberto Gonzales,"Gonzales, Alberto",100,"Gonzales, Alberto",30
5,Luis Manuel Martinez L.,"Martinez Lara, Luis Manuel",94,"Martinez Lara, Luis Manuel",20
6,Pedro Enrique,"Enrique, Pedro",100,"Enrique, Pedro",20
